# Estimating the full Stokes drift profile at Gotland

The surface Stokes drift is available at Gotland from the [CMEMS WAM hindcast of the Beltic Sea](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=BALTICSEA_ANALYSIS_FORECAST_WAV_003_010).
To estimate the Stokes drift profile, we can assume a profile shape based on some empirical wave spectrum (e.g., [Breivik et al, 2014](https://doi.org/10.1175/JPO-D-14-0020.1); [2016](https://doi.org/10.1016/j.ocemod.2016.01.005), [Li et al., 2017](https://doi.org/10.1016/j.ocemod.2017.03.016)).
Then we need to estimate the vertically integrated Stokes drift (or Stokes transport) from availble wave parameters. 

Availbale wave parameters:
- Significant wave height, $H_{m0}=4m_0^{1/2}$, where $m_0$ is the zeroth frequency moment of the wave spectrum.
- Mean period from the second frequency moment of the wave spectrum, $T_{m02}=(m_0/m_2)^{1/2}$

The Stokes transport is proportional to the first frequency moment of the wave spectrum,
$$
V^S = 2\pi c_1 m_1 = \pi c_1 H_{m0}^2/8 T_{m01},
$$
where $T_{m01}$ is the mean wave period from the first freqnency moment of the wave spectrum, $T_{m01}=m_0/m_1$, and $c_1\le1$ is a magnitude loss coefficent to accounting for the directional spreading effects.
Assuming the the Pierson–Moskowitz spectrum we have the relation between $T_{m01}\approx 1.086 T_{m02}$ ([Webb et al., 2011](https://doi.org/10.1016/j.ocemod.2011.08.007)).
Then we can follow the approach of [Li et al., 2017](https://doi.org/10.1016/j.ocemod.2017.03.016) to estimate the Stokes drift profile.
With better constraints of the surface Stokes drift and the significant wave height and mean period, we expect better estimation of the Stokes drift profile than [Li et al., 2017](https://doi.org/10.1016/j.ocemod.2017.03.016), in which both the surface Stokes drift and Stokes transport are estimated from the wind.

In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
from scipy import special
import matplotlib.pyplot as plt
sys.path.append("../gotmtool")
from gotmtool import *

### Load data

In [ ]:
datapath_wav = os.path.join(os.environ['HOME'], 'data', 'Baltic_Sea', 'GotlandBasin.waves.nc') 

In [ ]:
startdate = '1997-01-01'
enddate = '2002-01-01'

In [ ]:
ds_wav = xr.open_dataset(datapath_wav).sel(time=slice(startdate, enddate))

In [ ]:
time = ds_wav.time.values
ntime = time.size

In [ ]:
# wave variables
Hm0 = ds_wav.data_vars['VHM0'].values.squeeze()
Tm02 = ds_wav.data_vars['VTM02'].values.squeeze()
us0x = ds_wav.data_vars['VSDX'].values.squeeze()
us0y = ds_wav.data_vars['VSDY'].values.squeeze()
us0 = xr.ufuncs.sqrt(us0x**2+us0y**2)

### Vertical grid

In [ ]:
zi = -np.linspace(1, 32, 32)
nz = zi.size
dz = np.zeros_like(zi)
z = np.zeros(nz)
z[0] = 0.5*zi[0]
z[1:] = 0.5*(zi[:-1]+zi[1:])

### Compute Stokes drift profile

In [ ]:
# Stokes transport
c1 = 0.698
VS = np.pi*c1*Hm0**2/8./Tm02/1.086
ds = VS/us0
kp = 0.176*us0/VS
kps = 2.56*kp
def T1(k, z):
    return np.exp(2*k*z)
def T2(k, z):
    return np.sqrt(2.*np.pi*k*np.abs(z))*special.erfc(np.sqrt(2*k*np.abs(z)))
def stokes_drift_profile_srf(kp, kps, z):
    z0 = np.abs(z)
    r1 = (0.151/kp/z0-0.84)*(1.-T1(kp, -z0))
    r2 = -(0.84+0.0591/kp/z0)*T2(kp, z0)
    r3 = (0.0632/kps/z0+0.125)*(1.-T1(kps, -z0))
    r4 = (0.125+.0946/kps/z0)*T2(kps, z0)
    return 0.715+r1+r2+r3+r4
ustokes = np.zeros([ntime, nz])
vstokes = np.zeros([ntime, nz])
stokes_srf = np.zeros([ntime, nz])
for i in np.arange(nz):
    stokes_srf[:,i] = stokes_drift_profile_srf(kp, kps, zi[i])
ustokes[:,0] = us0x*stokes_srf[:,0]
vstokes[:,0] = us0y*stokes_srf[:,0]
for i in np.arange(1, nz):
    tmp = (stokes_srf[:,i-1]*zi[i-1]-stokes_srf[:,i]*zi[i])/(zi[i-1]-zi[i])
    ustokes[:,i] = us0x*tmp
    vstokes[:,i] = us0y*tmp



### Convert time from `numpy.datetime64` to `datetime.datetime`

In [ ]:
dttime = [pd.Timestamp(time[i]).to_pydatetime() for i in np.arange(ntime)]

### Save Stokes drift profile

In [ ]:
dat_dump_pfl(dttime, z, [ustokes, vstokes], 'us_prof.dat')

### Save surface Stokes drift and decay depth

In [ ]:
dat_dump_ts(dttime, [us0x, us0y, ds], 'us_surface.dat') 